In [15]:
import os

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
api_key = os.getenv("LANGCHAIN_API_KEY")
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
project = os.getenv("LANGCHAIN_PROJECT")


In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.0-pro",convert_system_message_to_human=True)

In [21]:
print(model.invoke('hi').content)

Hello there! How can I assist you today?


In [22]:
import bs4
from langchain import hub

In [23]:
from langchain.chains import create_retrieval_chain

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [25]:
from langchain_chroma import Chroma 

In [26]:
from langchain_community.document_loaders import WebBaseLoader

In [27]:
from langchain_core.prompts import ChatPromptTemplate

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [29]:
from langchain_core.prompts import MessagesPlaceholder

In [30]:
#Post request
# header, content, title will be retrieved from the web page
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)

In [31]:
# Load the doc from the webpage
doc = loader.load()

In [ ]:

doc